# Crop and Segment

Using a pre-trained model, crop into our image and produce segmented images for classification!

In [5]:
from pathlib import Path
import math

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.utils import load_img

from skimage.transform import resize
from skimage.io import imread, imshow, imsave

In [6]:
IMG_WIDTH_ORIGINAL=512
IMG_HEIGHT_ORIGINAL=1024
IMG_WIDTH = 256
IMG_HEIGHT = 512
IMG_CHANNELS = 1 # grayscale images
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)
NUM_CLASSES = 8 #8 in case just fluids is 4, in case fluids and layers is 8
BATCH_SIZE = 4 # try 4, 8, 12, 16, 32

SRF_CLASS = 6
IRF_CLASS = 7

CLASS_LABELS = (
    "Above ILM",
    "ILM-IPL/INL",
    "IPL/INL-RPE",
    "RPE-BM",
    "Under BM",
    "PED",
    "SRF",
    "IRF",
)

# Predict 💡

Make predictions on all of our image types

In [7]:
model = keras.models.load_model("../oct_model_20230125-092315/")

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-02-21 16:26:18.123495: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-21 16:26:18.123534: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
OCTID_DATA = Path("./data/OCTID/")

normal = OCTID_DATA.glob("**/NORMAL*")
armd = OCTID_DATA.glob("**/AMRD*.jpeg")
dr = OCTID_DATA.glob("**/DR*")
mh = OCTID_DATA.glob("**/MH*")
csr = OCTID_DATA.glob("**/CSR*")

In [9]:
normal_img_paths = [*normal]
armd_img_paths = [*armd]
dr_img_paths = [*dr]
mh_img_paths = [*mh]
csr_img_paths = [*csr]

In [10]:
Path('tmp/intermediate/normal_cropped').mkdir(parents=True)
Path('tmp/intermediate/armd_cropped').mkdir(parents=True)
Path('tmp/intermediate/dr_cropped').mkdir(parents=True)
Path('tmp/intermediate/mh_cropped').mkdir(parents=True)
Path('tmp/intermediate/csr_cropped').mkdir(parents=True)

In [11]:
for img_path in normal_img_paths:
    img = imread(img_path, as_gray=True)
    img = img[:, 142:-142]
    outpath = "tmp/intermediate/normal_cropped/" + str(img_path).split('/')[-1].split('.')[0] + '.png'
    imsave(outpath, img, check_contrast=False)

for img_path in armd_img_paths:
    img = imread(img_path, as_gray=True)
    img = img[:, 142:-142]
    outpath = "tmp/intermediate/armd_cropped/" + str(img_path).split('/')[-1].split('.')[0] + '.png'
    imsave(outpath, img, check_contrast=False)

for img_path in dr_img_paths:
    img = imread(img_path, as_gray=True)
    img = img[:, 142:-142]
    outpath = "tmp/intermediate/dr_cropped/" + str(img_path).split('/')[-1].split('.')[0] + '.png'
    imsave(outpath, img, check_contrast=False)

for img_path in mh_img_paths:
    img = imread(img_path, as_gray=True)
    img = img[:, 142:-142]
    outpath = "tmp/intermediate/mh_cropped/" + str(img_path).split('/')[-1].split('.')[0] + '.png'
    imsave(outpath, img, check_contrast=False)

for img_path in csr_img_paths:
    img = imread(img_path, as_gray=True)
    img = img[:, 142:-142]
    outpath = "tmp/intermediate/csr_cropped/" + str(img_path).split('/')[-1].split('.')[0] + '.png'
    imsave(outpath, img, check_contrast=False)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

In [12]:
class OCTIDSequence(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_train_paths = input_img_paths

    def __len__(self):
        return math.floor(len(self.input_train_paths) / self.batch_size)

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_train_paths = self.input_train_paths[i : i + self.batch_size]
        x = np.zeros((BATCH_SIZE,) + self.img_size + (1,), dtype="float32")
        for j, path in enumerate(batch_input_train_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            img=np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 1))
            x[j] = img/.255
        return x

In [13]:
OCTID_DATA = Path("./tmp/intermediate/")

normal = OCTID_DATA.glob("**/NORMAL*")
armd = OCTID_DATA.glob("**/AMRD*.jpeg")
dr = OCTID_DATA.glob("**/DR*")
mh = OCTID_DATA.glob("**/MH*")
csr = OCTID_DATA.glob("**/CSR*")

normal_img_paths = [*normal]
armd_img_paths = [*armd]
dr_img_paths = [*dr]
mh_img_paths = [*mh]
csr_img_paths = [*csr]

In [14]:
octid_seq = OCTIDSequence(BATCH_SIZE, IMG_SIZE, csr_img_paths)

In [15]:
preds = model.predict(octid_seq)
preds = np.argmax(preds, axis=-1)

2023-02-21 16:26:37.096362: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-21 16:26:37.246123: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


25/25 [==============================] - 6s 243ms/step


In [16]:
Path('segmentation/octid_predictions/csr/').mkdir(parents=True)
Path('segmentation/octid_predictions/mh/').mkdir(parents=True)
Path('segmentation/octid_predictions/armd/').mkdir(parents=True)
Path('segmentation/octid_predictions/normal/').mkdir(parents=True)
Path('segmentation/octid_predictions/dr/').mkdir(parents=True)

In [17]:
octid_seq = OCTIDSequence(BATCH_SIZE, IMG_SIZE, normal_img_paths)

preds = model.predict(octid_seq)
preds = np.argmax(preds, axis=-1)

for idx, image in enumerate(preds):
    # colour_image = utils.num_to_colors(image, IMG_HEIGHT, IMG_WIDTH)
    # extract name of file from path
    name = str(normal_img_paths[idx]).split("/")[-1].split(".")[0]
    imsave(f"./segmentation/octid_predictions/normal/{name}.png", image.astype(np.uint8))

51/51 [==============================] - 14s 279ms/step


/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/4112912699.py:10: UserWarning: ./segmentation/octid_predictions/normal/NORMAL148.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/normal/{name}.png", image.astype(np.uint8))
/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/4112912699.py:10: UserWarning: ./segmentation/octid_predictions/normal/NORMAL160.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/normal/{name}.png", image.astype(np.uint8))
/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/4112912699.py:10: UserWarning: ./segmentation/octid_predictions/normal/NORMAL174.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/normal/{name}.png", image.astype(np.uint8))
/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/4112912699.py:10: UserWarning: ./segmentation/octid_predictions/normal/NORMAL88.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/n

In [22]:
octid_seq = OCTIDSequence(BATCH_SIZE, IMG_SIZE, armd_img_paths)

preds = model.predict(octid_seq)
preds = np.argmax(preds, axis=-1)

for idx, image in enumerate(preds):
    # colour_image = utils.num_to_colors(image, IMG_HEIGHT, IMG_WIDTH)
    # extract name of file from path
    name = str(armd_img_paths[idx]).split("/")[-1].split(".")[0]
    imsave(f"./segmentation/octid_predictions/armd/{name}.png", image.astype(np.uint8))

ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [19]:
octid_seq = OCTIDSequence(BATCH_SIZE, IMG_SIZE, csr_img_paths)

preds = model.predict(octid_seq)
preds = np.argmax(preds, axis=-1)

for idx, image in enumerate(preds):
    # colour_image = utils.num_to_colors(image, IMG_HEIGHT, IMG_WIDTH)
    # extract name of file from path
    name = str(csr_img_paths[idx]).split("/")[-1].split(".")[0]
    imsave(f"./segmentation/octid_predictions/csr/{name}.png", image.astype(np.uint8))

25/25 [==============================] - 6s 260ms/step


/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/2145232654.py:10: UserWarning: ./segmentation/octid_predictions/csr/CSR101.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/csr/{name}.png", image.astype(np.uint8))
/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/2145232654.py:10: UserWarning: ./segmentation/octid_predictions/csr/CSR12.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/csr/{name}.png", image.astype(np.uint8))
/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/2145232654.py:10: UserWarning: ./segmentation/octid_predictions/csr/CSR13.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/csr/{name}.png", image.astype(np.uint8))
/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/2145232654.py:10: UserWarning: ./segmentation/octid_predictions/csr/CSR100.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/csr/{name}.png", image.astype(np.ui

In [20]:
octid_seq = OCTIDSequence(BATCH_SIZE, IMG_SIZE, dr_img_paths)

preds = model.predict(octid_seq)
preds = np.argmax(preds, axis=-1)

for idx, image in enumerate(preds):
    # colour_image = utils.num_to_colors(image, IMG_HEIGHT, IMG_WIDTH)
    # extract name of file from path
    name = str(dr_img_paths[idx]).split("/")[-1].split(".")[0]
    imsave(f"./segmentation/octid_predictions/dr/{name}.png", image.astype(np.uint8))
    

26/26 [==============================] - 7s 260ms/step


/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/3146808694.py:10: UserWarning: ./segmentation/octid_predictions/dr/DR28.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/dr/{name}.png", image.astype(np.uint8))
/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/3146808694.py:10: UserWarning: ./segmentation/octid_predictions/dr/DR14.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/dr/{name}.png", image.astype(np.uint8))
/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/3146808694.py:10: UserWarning: ./segmentation/octid_predictions/dr/DR15.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/dr/{name}.png", image.astype(np.uint8))
/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/3146808694.py:10: UserWarning: ./segmentation/octid_predictions/dr/DR29.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/dr/{name}.png", image.astype(np.uint8))
/var/fol

In [21]:
octid_seq = OCTIDSequence(BATCH_SIZE, IMG_SIZE, mh_img_paths)

preds = model.predict(octid_seq)
preds = np.argmax(preds, axis=-1)

for idx, image in enumerate(preds):
    # colour_image = utils.num_to_colors(image, IMG_HEIGHT, IMG_WIDTH)
    # extract name of file from path
    name = str(mh_img_paths[idx]).split("/")[-1].split(".")[0]
    imsave(f"./segmentation/octid_predictions/mh/{name}.png", image.astype(np.uint8))
    

25/25 [==============================] - 6s 240ms/step


/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/1335466976.py:10: UserWarning: ./segmentation/octid_predictions/mh/MH87.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/mh/{name}.png", image.astype(np.uint8))
/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/1335466976.py:10: UserWarning: ./segmentation/octid_predictions/mh/MH93.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/mh/{name}.png", image.astype(np.uint8))
/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/1335466976.py:10: UserWarning: ./segmentation/octid_predictions/mh/MH44.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/mh/{name}.png", image.astype(np.uint8))
/var/folders/3p/cmcfy1q16ts0p3tb67072b6c0000gn/T/ipykernel_1352/1335466976.py:10: UserWarning: ./segmentation/octid_predictions/mh/MH50.png is a low contrast image
  imsave(f"./segmentation/octid_predictions/mh/{name}.png", image.astype(np.uint8))
/var/fol

# Cleanup! 🧻

Finally, cleanup our dirty tmp files

In [23]:
def rmtree(f: Path):
    if f.is_file():
        f.unlink()
    else:
        for child in f.iterdir():
            rmtree(child)
        f.rmdir()

rmtree(Path('./tmp/'))

Clean up predicted images if so required as well

In [4]:
predicted_paths = [Path('segmentation/octid_predictions/csr/'), Path('segmentation/octid_predictions/mh/'), Path('segmentation/octid_predictions/armd/'), Path('segmentation/octid_predictions/normal/'), Path('segmentation/octid_predictions/dr/')]

for path in predicted_paths:
    rmtree(path)